In [1]:
import pandas
import numpy as np

In [2]:
All_probabilities = pandas.read_hdf('/home/amachlou/Documents/INRIA/Sulcal_Neuroanatomy/all_anterior_inferior_probabilities_RL_only_common_subjects.hdf')
All_probabilities = All_probabilities.rename({'inferior':'superior'})
All_probabilities

sulcus                                                      Lat_Fis-ant-Horizont  \
hemisphere sulcus                   probability_of Subject                         
left       Lat_Fis-ant-Horizont     anterior       102715                    NaN   
                                    superior       102715                    NaN   
           Lat_Fis-ant-Vertical     anterior       102715           7.816540e-06   
                                    superior       102715           9.966060e-01   
           Lat_Fis-post             anterior       102715           0.000000e+00   
                                    superior       102715           9.996392e-01   
           S_calcarine              anterior       102715           0.000000e+00   
                                    superior       102715           7.433579e-07   
           S_central                anterior       102715           0.000000e+00   
                                    superior       102715           1.000000e+00   
           S_cingul-Marginalis      anterior       102715           0.000000e+00   
                                    superior       102715           1.000000e+00   
           S_circular_insula_ant    anterior       102715           5.195230e-02   
                                    superior       102715           5.186942e-03   
           S_circular_insula_inf    anterior       102715           2.403413e-09   
                                    superior       102715           1.022933e-01   
           S_circular_insula_sup    anterior       102715           4.297712e-02   
                                    superior       102715           9.666300e-01   
           S_collat_transv_ant      anterior       102715           0.000000e+00   
                                    superior       102715           0.000000e+00   
           S_collat_transv_post     anterior       102715           0.000000e+00   
                                    superior       102715           0.000000e+00   
           S_front_inf              anterior       102715           2.972033e-01   
                                    superior       102715           9.999185e-01   
           S_front_middle           anterior       102715           8.453276e-01   
                                    superior       102715           9.999996e-01   
           S_front_sup              anterior       102715           2.211952e-01   
                                    superior       102715           1.000000e+00   
           S_interm_prim-Jensen     anterior       102715           0.000000e+00   
                                    superior       102715           1.000000e+00   
...                                                                          ...   
right      S_oc_sup_and_transversal anterior       994273           0.000000e+00   
                                    superior       994273           9.903859e-01   
           S_occipital_ant          anterior       994273           0.000000e+00   
                                    superior       994273           9.397115e-02   
           S_orbital-H_Shaped       anterior       994273           8.051229e-01   
                                    superior       994273           0.000000e+00   
           S_orbital_lateral        anterior       994273           9.999812e-01   
                                    superior       994273           4.105779e-01   
           S_orbital_med-olfact     anterior       994273           4.528448e-01   
                                    superior       994273           0.000000e+00   
           S_parieto_occipital      anterior       994273           0.000000e+00   
                                    superior       994273           9.999995e-01   
           S_pericallosal           anterior       994273           9.539183e-02   
                                    superior       994273           8.812661e-01   
           S_postcentral            anterior       9

In [3]:
hemispheres = list(All_probabilities.index.levels[0])
subject_ids = list(All_probabilities.index.levels[3])
sulci_names = All_probabilities.columns

In [4]:
def making_tables_of_each_sub_and_hem(all_probs, subject_ids, hemispheres):
    '''
    Input: pandas dataframe table of all subjects sulcal relations, both hemispheres
    Retrieves information of each subject's and each hemisphere's probabilities, and
    creates a dataframe table per subject per hemisphere
    '''
    subjects = {}
    
    for subject in subject_ids:
        hemisphere = {}
        for hem in hemispheres:
            sub_and_hem = all_probs.loc[(hem, slice(None), slice(None), subject)]
            hemisphere[hem]= sub_and_hem
        subjects[subject]=hemisphere
    
    return subjects
            
    

In [5]:
all_subs_and_hems = making_tables_of_each_sub_and_hem(All_probabilities, subject_ids, hemispheres)

In [6]:
def making_sets_of_relative_sulci_per_hemisphere_and_subject(all_subs_and_hems, sulci_names):
    '''
    Creates a quadruple dictionary with keys: subject number, hemisphere, reference sulcus, relative position
    
    all_sulci_sets[subject][hemisphere][reference_sulcus][relative_position]
    
    Parameters: all_subs_and_hems: dictionary with keys: subject number, hemisphere
                sulci_names: pandas dataframe index array
                             names of sulci in one hemisphere
    Returns: all_sulci_sets: dict
    
    '''
    
    all_sulci_sets = {}
    
    for subject in all_subs_and_hems:
        
        hemisphere = {}
        
        for hem in all_subs_and_hems[subject]:
            
            sulci_sets = {}

            for sulcus_name in sulci_names:
               
                current_sub_and_hem = all_subs_and_hems[subject][hem]
            
                anterior_probability = current_sub_and_hem.xs('anterior', level='probability_of')
                superior_probability = current_sub_and_hem.xs('superior', level='probability_of')
                anterior_sulci=set(anterior_probability[sulcus_name][(anterior_probability>0.95)[sulcus_name]].index)
                posterior_sulci=set(anterior_probability[sulcus_name][(anterior_probability<0.05)[sulcus_name]].index)
                superior_sulci=set(superior_probability[sulcus_name][(superior_probability>0.95)[sulcus_name]].index)
                inferior_sulci=set(superior_probability[sulcus_name][(superior_probability<0.05)[sulcus_name]].index)
                position = {}
                position['anterior']=anterior_sulci
                position['posterior']=posterior_sulci
                position['superior']=superior_sulci
                position['inferior']=inferior_sulci
                sulci_sets[sulcus_name]=position
            
            hemisphere[hem]=sulci_sets
        
        all_sulci_sets[subject]=hemisphere
    
    return all_sulci_sets

In [7]:
All_subject_and_hemisphere_sets = making_sets_of_relative_sulci_per_hemisphere_and_subject(all_subs_and_hems, sulci_names)

### Making sulci formulae

In [8]:
for a in range(len(subject_ids)):
        subject = subject_ids[a]

In [9]:
formula_for_prcs = (All_subject_and_hemisphere_sets[subject]['left']["S_central"]['anterior']).intersection(
    All_subject_and_hemisphere_sets[subject]['left']["Lat_Fis-ant-Horizont"]['superior'].union(
        All_subject_and_hemisphere_sets[subject]['left']["Lat_Fis-ant-Vertical"]['superior'].union(
            All_subject_and_hemisphere_sets[subject]['left']["Lat_Fis-post"]['superior']
        )
    )
)

In [10]:
formula_for_sfs = ((All_subject_and_hemisphere_sets[subject]['left']["S_precentral-sup-part"]['anterior']).union(
    All_subject_and_hemisphere_sets[subject]['left']["S_precentral-inf-part"]['anterior'])).intersection(
    All_subject_and_hemisphere_sets[subject]['left']["S_front_inf"]['superior']
)

In [11]:
formula_for_calcarine = (All_subject_and_hemisphere_sets[subject]['left']['S_parieto_occipital']['inferior'].intersection(
    All_subject_and_hemisphere_sets[subject]['left']['S_oc_middle_and_Lunatus']['anterior']).intersection(
        All_subject_and_hemisphere_sets[subject]['left']['Lat_Fis-post']['posterior']).intersection(
            All_subject_and_hemisphere_sets[subject]['left']['S_occipital_ant']['superior']))

In [12]:
formula_for_pocs = ((All_subject_and_hemisphere_sets[subject]['left']['S_parieto_occipital']['anterior']).intersection((
    All_subject_and_hemisphere_sets[subject]['left']['Lat_Fis-ant-Horizont']['superior'].union(
        All_subject_and_hemisphere_sets[subject]['left']['Lat_Fis-ant-Vertical']['superior'].union(
            All_subject_and_hemisphere_sets[subject]['left']['Lat_Fis-post']['superior']))).intersection(
                All_subject_and_hemisphere_sets[subject]['left']['S_central']['posterior'])))

In [13]:
formula_for_ifs = All_subject_and_hemisphere_sets[subject]['left']['S_front_sup']['inferior'].intersection(
    All_subject_and_hemisphere_sets[subject]['left']['S_precentral-sup-part']['anterior'])

In [14]:
formula_for_pericallosal = All_subject_and_hemisphere_sets[subject]['left']['S_central']['inferior'].intersection(
    All_subject_and_hemisphere_sets[subject]['left']['S_cingul-Marginalis']['inferior']).intersection(
        All_subject_and_hemisphere_sets[subject]['left']['S_calcarine']['anterior']).intersection(
            All_subject_and_hemisphere_sets[subject]['left']['Lat_Fis-ant-Horizont']['posterior']).intersection(
                All_subject_and_hemisphere_sets[subject]['left']['S_circular_insula_inf']['superior'])

In [15]:
formula_for_los = All_subject_and_hemisphere_sets[subject]['left']['S_parieto_occipital']['inferior'].intersection(
    All_subject_and_hemisphere_sets[subject]['left']['Lat_Fis-post']['inferior']).intersection(
       All_subject_and_hemisphere_sets[subject]['left']['S_temporal_transverse']['posterior'])

In [16]:
formula_for_sos = All_subject_and_hemisphere_sets[subject]['left']['S_intrapariet_and_P_trans']['posterior'].intersection(
    All_subject_and_hemisphere_sets[subject]['left']['S_oc_middle_and_Lunatus']['superior'])

In [17]:
formula_for_subparietal = All_subject_and_hemisphere_sets[subject]['left']['S_parieto_occipital']['superior'].intersection(
    All_subject_and_hemisphere_sets[subject]['left']['S_postcentral']['posterior']).intersection(
    All_subject_and_hemisphere_sets[subject]['left']['S_cingul-Marginalis']['inferior'])

In [18]:
LF = (All_subject_and_hemisphere_sets[subject]['left']["Lat_Fis-ant-Horizont"]['superior'].union(
        All_subject_and_hemisphere_sets[subject]['left']["Lat_Fis-ant-Vertical"]['superior']).union(
            All_subject_and_hemisphere_sets[subject]['left']["Lat_Fis-post"]['superior']))

In [19]:
formula_for_sts = (All_subject_and_hemisphere_sets[subject]['left']["Lat_Fis-ant-Horizont"]['posterior'].union(
        All_subject_and_hemisphere_sets[subject]['left']["Lat_Fis-ant-Vertical"]['posterior']).union(
            All_subject_and_hemisphere_sets[subject]['left']["Lat_Fis-post"]['inferior'])).intersection(
                All_subject_and_hemisphere_sets[subject]['left']['Lat_Fis-post']['posterior'])

# Hard-Coding Sulcal Probabilities

### Code for success of finding sulcus

In [20]:
def calculating_prob_of_sulcus_found(subject_ids, sulci_output, desired_sulcus, destrieux_name):
    '''
    Input: subject_ids=list of subject ids, 
            sulci_output=dictionary, key: subject id, value: names of sulci returned from formula to find one sulcus
    Function iterates through sulci_output dictionary to check if desired sulcus is found in subject or not.
    Output: Total number of subjects for which desired sulcus was found
            Total number of subjects
            Probability of formula returning the desired sulcus within a subject.
    '''
    
    desired_sulcus = []

    for current_subject in subject_ids:
        check_for_sulcus = destrieux_name in sulci_output[current_subject]
        desired_sulcus.append(check_for_sulcus)

    prob_of_sulcus_found = (sum(desired_sulcus))/(len(desired_sulcus))

    return sum(desired_sulcus), len(desired_sulcus), prob_of_sulcus_found

In [21]:
def accuracy_check(subject_ids, sulcus_check, destrieux_name):
    '''
    Input: dictionary with subject id as key, list of found sulci from sulcal formula as values
    Function searches for desired sulcus within values, and if found, 
    Returns: dictionary of subject for which search was successful, with length of list of sulci found
            : Number of subjects for which only desired sulcus was found
            : Proportion of accuracy
    '''
    
    sensitivity_output = {}
    accuracy_of_output = 0
    
    for current_subject in subject_ids:
        if destrieux_name in sulcus_check[current_subject]:
            sensitivity_output[current_subject]=len(sulcus_check[current_subject])
            if len(sulcus_check[current_subject]) is 1:
                accuracy_of_output +=1
    
    return  sensitivity_output, accuracy_of_output, accuracy_of_output / (len(sensitivity_output))

## Sulci analysed (20)

##### Precentral sulcus, Marginal sulcus, Intraparietal sulcus, Collateral sulcus (posterior), Collateral sulcus (anterior), Frontal middle sulcus, Temporal transverse sulcus, Inferior temporal sulcus, Superior temporal sulcus, Subparietal sulcus, Superior frontal sulcus, Calcarine sulcus, Postcentral sulcus, Inferior frontal sulcus, Pericallosal sulcus, Lateral occipit(o-temporal) sulcus, Suborbital sulcus, Sulcus intermedius primus of Jensen, Occipital middle and lunatus sulcus, Superior occipital sulcus

### Precentral Sulcus

In [22]:
def prcs_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = (current_subject_hemi["S_central"]['anterior']).intersection(
        current_subject_hemi["Lat_Fis-ant-Horizont"]['superior'].union(
            current_subject_hemi["Lat_Fis-ant-Vertical"]['superior'].union(
                current_subject_hemi["Lat_Fis-post"]['superior']
                )
            )
        )

        sulci_found[current_subject]=sulcus_test

    return sulci_found

In [23]:
PrCS_check_left = prcs_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
PrCS_check_right = prcs_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [24]:
def calculating_prob_of_PrCS_found(subject_ids, sulci_output, desired_sulcus):
    '''
    Input: subject_ids=list of subject ids, 
            sulci_output=dictionary, key: subject id, value: names of sulci returned from formula to find one sulcus
    Function iterates through sulci_output dictionary to check if desired sulcus is found in subject or not.
    Output: Total number of subjects for which desired sulcus was found
            Total number of subjects
            Probability of formula returning the desired sulcus within a subject.
    '''
    
    desired_sulcus = []

    for current_subject in subject_ids:
        check_for_sulcus = 'S_precentral-inf-part' in sulci_output[current_subject] and 'S_precentral-sup-part' in sulci_output[current_subject]
        desired_sulcus.append(check_for_sulcus)

    prob_of_sulcus_found = (sum(desired_sulcus))/(len(desired_sulcus))

    return sum(desired_sulcus), len(desired_sulcus), prob_of_sulcus_found

In [25]:
PrCS_prob_left = calculating_prob_of_PrCS_found(subject_ids, PrCS_check_left, 'Precentral_sulcus')
PrCS_prob_right = calculating_prob_of_PrCS_found(subject_ids, PrCS_check_right, 'Precentral_sulcus')

In [26]:
print(PrCS_prob_left, PrCS_prob_right)

(262, 572, 0.458041958041958) (245, 572, 0.42832167832167833)


In [27]:
def PrCS_formula_accuracy_check(subject_ids, sulcus_check):
    '''
    Input: dictionary with subject id as key, list of found sulci from sulcal formula as values
    Function searches for desired sulcus within values, and if found, 
    Returns: dictionary of subject for which search was successful, with length of list of sulci found
            : Number of subjects for which only desired sulcus was found
            : Proportion of accuracy
    '''
    
    sensitivity_output = {}
    accuracy_of_output = 0
    
    for current_subject in subject_ids:
        if 'S_precentral-sup-part' in sulcus_check[current_subject] and 'S_precentral-inf-part' in sulcus_check[current_subject]:
            sensitivity_output[current_subject]=len(sulcus_check[current_subject])
            if len(sulcus_check[current_subject]) is 1:
                accuracy_of_output +=1
    
    return  sensitivity_output, accuracy_of_output, accuracy_of_output / (len(sensitivity_output))

In [28]:
PrCS_left_accuracy = PrCS_formula_accuracy_check(subject_ids, PrCS_check_left)
PrCS_right_accuracy = PrCS_formula_accuracy_check(subject_ids, PrCS_check_right)

In [29]:
PrCS_left_accuracy

({102715: 6,
  102816: 5,
  103111: 6,
  103212: 6,
  103414: 4,
  104416: 7,
  104820: 6,
  105115: 5,
  106319: 5,
  106521: 7,
  107018: 6,
  108525: 7,
  108828: 5,
  109123: 6,
  110411: 7,
  111312: 6,
  111413: 7,
  112112: 6,
  112819: 6,
  113619: 5,
  113821: 6,
  113922: 3,
  114419: 6,
  115017: 7,
  116524: 7,
  117324: 6,
  118124: 7,
  120717: 4,
  121315: 6,
  121416: 6,
  121618: 5,
  122620: 5,
  123117: 7,
  123420: 5,
  123925: 6,
  125525: 7,
  126931: 5,
  127630: 6,
  127832: 6,
  127933: 7,
  131217: 7,
  131419: 5,
  131621: 7,
  131823: 5,
  133625: 6,
  134021: 6,
  134324: 6,
  135124: 6,
  135225: 6,
  135528: 6,
  137431: 6,
  138534: 7,
  139435: 6,
  139839: 6,
  140420: 5,
  140925: 6,
  144226: 7,
  145632: 7,
  150524: 5,
  150625: 5,
  150726: 7,
  151223: 7,
  151425: 6,
  151930: 6,
  152225: 6,
  153025: 5,
  154835: 6,
  155231: 5,
  155635: 7,
  156031: 6,
  156233: 6,
  157336: 6,
  158843: 5,
  161630: 6,
  167440: 6,
  167743: 6,
  168139: 4,

In [30]:
PrCS_right_accuracy

({103111: 6,
  103212: 3,
  103515: 6,
  104012: 6,
  105115: 5,
  105216: 5,
  105620: 5,
  106319: 5,
  106521: 5,
  108121: 3,
  108323: 5,
  109123: 5,
  110411: 5,
  111413: 4,
  111716: 3,
  113215: 3,
  113619: 5,
  113922: 6,
  114419: 4,
  117122: 5,
  117324: 4,
  117930: 6,
  118124: 4,
  120717: 5,
  123420: 5,
  124220: 6,
  124826: 5,
  126931: 5,
  127832: 5,
  127933: 5,
  131722: 3,
  131823: 6,
  133019: 5,
  133625: 5,
  133827: 5,
  134324: 6,
  135124: 4,
  135528: 4,
  136833: 3,
  137431: 4,
  139839: 6,
  140420: 5,
  140925: 4,
  141826: 6,
  142424: 5,
  144226: 6,
  145632: 4,
  148133: 4,
  148941: 6,
  150524: 6,
  151425: 5,
  151627: 6,
  152225: 7,
  153025: 7,
  153429: 5,
  153833: 5,
  154431: 6,
  154734: 3,
  154936: 4,
  155231: 6,
  156637: 5,
  157336: 5,
  158540: 5,
  158843: 4,
  161327: 5,
  162733: 6,
  163129: 6,
  163331: 6,
  165032: 7,
  167440: 6,
  167743: 5,
  168139: 4,
  169444: 5,
  169747: 5,
  170934: 3,
  171330: 5,
  171633: 6,

### Marginal Sulcus

In [31]:
def marginal_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = current_subject_hemi['S_pericallosal']['superior'].intersection(
            current_subject_hemi['S_postcentral']['anterior'])
        
        sulci_found[current_subject]=sulcus_test

        
    return sulci_found

In [32]:
marginal_check_left = marginal_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
marginal_check_right = marginal_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [33]:
marginal_prob_left = calculating_prob_of_sulcus_found(subject_ids, marginal_check_left, 'Marginal_sulcus', 'S_cingul-Marginalis')
marginal_prob_right = calculating_prob_of_sulcus_found(subject_ids, marginal_check_right, 'Marginal_sulcus', 'S_cingul-Marginalis')

In [34]:
print(marginal_prob_left, marginal_prob_right)

(53, 572, 0.09265734265734266) (47, 572, 0.08216783216783216)


In [35]:
marginal_left_accuracy = accuracy_check(subject_ids, marginal_check_left, 'S_cingul-Marginalis')
marginal_right_accuracy = accuracy_check(subject_ids, marginal_check_right, 'S_cingul-Marginalis')

In [36]:
marginal_left_accuracy

({108828: 8,
  111413: 9,
  113821: 8,
  116524: 6,
  117122: 10,
  124220: 8,
  130619: 6,
  131621: 7,
  134324: 8,
  137027: 6,
  140925: 8,
  148941: 6,
  151728: 8,
  151930: 8,
  152427: 8,
  153126: 7,
  165032: 9,
  178849: 8,
  181232: 9,
  188145: 10,
  195041: 5,
  201111: 8,
  201414: 7,
  201818: 8,
  209329: 6,
  211720: 7,
  213522: 8,
  214019: 6,
  217429: 8,
  268850: 7,
  303624: 9,
  333330: 9,
  355542: 7,
  397861: 6,
  433839: 5,
  459453: 8,
  500222: 9,
  541640: 7,
  553344: 8,
  592455: 8,
  671855: 9,
  680452: 6,
  685058: 7,
  687163: 9,
  765056: 8,
  770352: 9,
  773257: 8,
  786569: 6,
  837964: 7,
  857263: 7,
  865363: 6,
  872764: 6,
  958976: 6},
 0,
 0.0)

In [37]:
marginal_right_accuracy

({103111: 7,
  104416: 8,
  107018: 8,
  108323: 9,
  112112: 9,
  122317: 7,
  125222: 9,
  126931: 10,
  151223: 10,
  153126: 6,
  153227: 9,
  156031: 9,
  173334: 8,
  173435: 8,
  180937: 7,
  196750: 8,
  201111: 6,
  204016: 7,
  209329: 9,
  211720: 11,
  221319: 6,
  285446: 8,
  346137: 8,
  358144: 9,
  415837: 7,
  459453: 6,
  480141: 10,
  497865: 9,
  500222: 7,
  530635: 9,
  559053: 7,
  580044: 9,
  580650: 8,
  588565: 7,
  628248: 9,
  685058: 7,
  731140: 9,
  763557: 9,
  765056: 7,
  771354: 9,
  782157: 7,
  825048: 6,
  844961: 7,
  859671: 10,
  861456: 4,
  910241: 7,
  958976: 8},
 0,
 0.0)

### Intraparietal Sulcus

In [38]:
def ips_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = current_subject_hemi['S_central']['posterior'].intersection(
            current_subject_hemi['S_parieto_occipital']['anterior']).intersection(
                current_subject_hemi['S_interm_prim-Jensen']['superior'])
        
        sulci_found[current_subject]=sulcus_test

        
    return sulci_found

In [39]:
ips_check_left = ips_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
ips_check_right = ips_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [40]:
ips_prob_left = calculating_prob_of_sulcus_found(subject_ids, ips_check_left, 'Intraparietal_sulcus', 'S_intrapariet_and_P_trans')
ips_prob_right = calculating_prob_of_sulcus_found(subject_ids, ips_check_right, 'Intraparietal_sulcus', 'S_intrapariet_and_P_trans')

In [41]:
print(ips_prob_left, ips_prob_right)

(42, 572, 0.07342657342657342) (16, 572, 0.027972027972027972)


In [42]:
ips_left_accuracy = accuracy_check(subject_ids, ips_check_left, 'S_intrapariet_and_P_trans')
ips_right_accuracy = accuracy_check(subject_ids, ips_check_right, 'S_intrapariet_and_P_trans')

In [43]:
ips_left_accuracy

({105014: 1,
  109123: 2,
  112819: 1,
  113922: 2,
  124826: 1,
  130922: 2,
  131722: 1,
  132118: 1,
  135528: 1,
  146533: 2,
  153126: 2,
  158136: 1,
  161630: 1,
  168341: 3,
  169343: 1,
  173536: 2,
  173738: 1,
  176037: 1,
  182739: 2,
  185442: 1,
  185947: 2,
  188145: 4,
  201414: 1,
  204218: 1,
  204521: 2,
  207123: 1,
  209329: 1,
  250427: 1,
  251833: 1,
  255639: 2,
  280941: 1,
  304020: 2,
  380036: 2,
  436845: 3,
  459453: 1,
  541640: 1,
  541943: 2,
  566454: 2,
  683256: 2,
  690152: 2,
  861456: 2,
  957974: 1},
 22,
 0.5238095238095238)

In [44]:
ips_right_accuracy

({111312: 1,
  113821: 1,
  127832: 2,
  144226: 2,
  148133: 1,
  153833: 1,
  156233: 1,
  168947: 2,
  198350: 2,
  211720: 1,
  249947: 4,
  280739: 2,
  732243: 2,
  802844: 2,
  820745: 1,
  896879: 1},
 8,
 0.5)

### Collateral Transverse Sulcus_Posterior

In [45]:
def coll_transv_post_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = current_subject_hemi['S_temporal_inf']['inferior']
        
        sulci_found[current_subject]=sulcus_test

        
    return sulci_found

In [46]:
collat_transv_post_check_left = coll_transv_post_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
collat_transv_post_check_right = coll_transv_post_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [47]:
collat_transv_post_prob_left = calculating_prob_of_sulcus_found(subject_ids, collat_transv_post_check_left, 'Posterior_collateral_transverse_sulcus', 'S_collat_transv_post')
collat_transv_post_prob_right = calculating_prob_of_sulcus_found(subject_ids, collat_transv_post_check_right, 'Posterior_collateral_transverse_sulcus', 'S_collat_transv_post')

In [48]:
print(collat_transv_post_prob_left, collat_transv_post_prob_right)

(213, 572, 0.3723776223776224) (264, 572, 0.46153846153846156)


In [49]:
collat_transv_post_left_accuracy = accuracy_check(subject_ids, collat_transv_post_check_left, 'S_collat_transv_post')
collat_transv_post_right_accuracy = accuracy_check(subject_ids, collat_transv_post_check_right, 'S_collat_transv_post')

In [50]:
collat_transv_post_left_accuracy

({102816: 1,
  103111: 1,
  103818: 1,
  105115: 1,
  105216: 1,
  105620: 1,
  108525: 1,
  109325: 2,
  110411: 1,
  111009: 1,
  112112: 1,
  113215: 1,
  113821: 1,
  114419: 1,
  114924: 1,
  115017: 1,
  116120: 1,
  116524: 1,
  117930: 1,
  121315: 1,
  121618: 1,
  123117: 1,
  123925: 1,
  124422: 1,
  125222: 1,
  125424: 1,
  127630: 1,
  127933: 1,
  130619: 1,
  131924: 1,
  132118: 1,
  137027: 1,
  137431: 1,
  138534: 1,
  143325: 2,
  144226: 1,
  146533: 1,
  148133: 1,
  150524: 1,
  150625: 1,
  150726: 1,
  151627: 1,
  151728: 1,
  151930: 1,
  152831: 1,
  153025: 1,
  153227: 1,
  154532: 1,
  154936: 1,
  156637: 1,
  158136: 1,
  159239: 1,
  163331: 1,
  165032: 1,
  167238: 1,
  168341: 1,
  168947: 1,
  169141: 1,
  169343: 1,
  169747: 1,
  170934: 2,
  171330: 1,
  172029: 1,
  172938: 1,
  173536: 1,
  175237: 1,
  176037: 1,
  177645: 1,
  178142: 2,
  178748: 1,
  178849: 1,
  179245: 1,
  180937: 1,
  181232: 1,
  182739: 1,
  185038: 1,
  185442: 2,

In [51]:
collat_transv_post_right_accuracy

({103111: 1,
  103212: 1,
  103515: 2,
  105620: 1,
  106319: 1,
  106521: 1,
  108525: 1,
  108828: 1,
  109325: 1,
  110411: 1,
  111009: 1,
  111716: 1,
  114419: 1,
  114924: 2,
  116120: 1,
  117930: 1,
  118225: 1,
  121416: 1,
  121618: 1,
  123117: 2,
  123420: 1,
  124826: 2,
  125222: 1,
  126931: 1,
  127630: 1,
  127832: 1,
  127933: 1,
  131621: 1,
  131823: 1,
  133827: 1,
  135932: 1,
  136227: 1,
  137027: 2,
  138534: 2,
  140420: 1,
  140925: 1,
  142424: 2,
  144226: 1,
  145531: 1,
  146533: 1,
  148941: 1,
  150625: 1,
  150726: 1,
  151526: 1,
  151627: 1,
  151728: 2,
  152831: 1,
  153126: 1,
  153429: 1,
  154431: 1,
  154532: 2,
  154734: 1,
  154835: 1,
  154936: 1,
  155231: 1,
  155635: 1,
  156031: 1,
  156233: 2,
  156637: 1,
  157336: 1,
  158136: 1,
  161327: 2,
  161731: 1,
  162228: 1,
  162329: 1,
  162733: 1,
  163331: 1,
  167238: 1,
  167440: 1,
  168341: 1,
  168947: 1,
  169141: 1,
  169343: 1,
  169747: 1,
  171330: 1,
  171431: 1,
  172130: 1,

### Collateral Transverse Sulcus_Anterior

In [52]:
def coll_transv_ant_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = current_subject_hemi['S_oc-temp_lat']['inferior']
        
        sulci_found[current_subject]=sulcus_test

        
    return sulci_found

In [53]:
collat_transv_ant_check_left = coll_transv_ant_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
collat_transv_ant_check_right = coll_transv_ant_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [54]:
collat_transv_ant_prob_left = calculating_prob_of_sulcus_found(subject_ids, collat_transv_ant_check_left, 'Anterior_collateral_transverse_sulcus', 'S_collat_transv_ant')
collat_transv_ant_prob_right = calculating_prob_of_sulcus_found(subject_ids, collat_transv_ant_check_right, 'Anterior_collateral_transverse_sulcus', 'S_collat_transv_ant')

In [55]:
print(collat_transv_ant_prob_left, collat_transv_ant_prob_right)

(170, 572, 0.2972027972027972) (154, 572, 0.2692307692307692)


In [56]:
collat_transv_ant_left_accuracy = accuracy_check(subject_ids, collat_transv_ant_check_left, 'S_collat_transv_ant')
collat_transv_ant_right_accuracy = accuracy_check(subject_ids, collat_transv_ant_check_right, 'S_collat_transv_ant')

In [57]:
collat_transv_ant_left_accuracy

({102715: 1,
  103212: 1,
  103414: 2,
  106319: 2,
  109123: 1,
  111413: 1,
  112819: 1,
  114419: 1,
  115017: 1,
  116120: 1,
  116524: 2,
  117930: 2,
  118225: 1,
  121315: 1,
  122317: 2,
  124826: 1,
  127832: 3,
  130619: 1,
  131722: 1,
  133019: 1,
  133625: 1,
  133827: 1,
  135124: 1,
  135528: 1,
  135932: 2,
  138534: 2,
  139435: 1,
  148941: 1,
  150524: 1,
  151223: 1,
  151627: 2,
  152831: 1,
  154431: 1,
  154835: 1,
  154936: 1,
  156233: 1,
  156637: 1,
  157437: 1,
  158035: 1,
  158136: 1,
  158843: 1,
  159239: 2,
  161327: 1,
  162329: 1,
  162733: 1,
  163129: 1,
  168947: 2,
  169343: 1,
  169747: 1,
  171330: 1,
  173132: 1,
  173536: 1,
  174437: 2,
  175035: 1,
  175237: 1,
  175742: 2,
  176542: 1,
  176744: 2,
  177645: 3,
  178748: 1,
  179548: 1,
  181131: 1,
  183034: 1,
  185139: 1,
  185947: 1,
  187547: 3,
  197550: 2,
  198855: 1,
  201414: 2,
  204218: 1,
  205220: 1,
  206222: 1,
  207123: 1,
  207628: 1,
  208024: 1,
  208327: 1,
  209834: 1,

In [58]:
collat_transv_ant_right_accuracy

({103111: 1,
  103212: 1,
  103414: 1,
  105014: 2,
  108323: 1,
  109123: 1,
  113821: 1,
  114419: 1,
  117930: 1,
  118225: 1,
  121315: 1,
  123420: 1,
  127630: 1,
  127832: 4,
  130619: 2,
  130922: 1,
  131217: 1,
  131419: 1,
  131823: 1,
  133019: 1,
  133827: 1,
  134324: 1,
  135932: 4,
  136833: 1,
  137027: 1,
  137431: 1,
  139435: 1,
  140925: 1,
  141826: 1,
  148133: 1,
  148941: 1,
  151223: 1,
  151425: 1,
  151627: 1,
  154431: 1,
  154835: 1,
  156031: 1,
  158035: 1,
  162733: 1,
  167440: 1,
  167743: 1,
  169343: 1,
  169747: 3,
  171330: 1,
  171431: 1,
  172130: 1,
  172534: 3,
  173536: 1,
  177746: 1,
  178142: 1,
  178748: 1,
  182032: 1,
  182739: 1,
  183034: 1,
  185038: 1,
  185947: 1,
  186141: 1,
  191336: 1,
  192540: 1,
  195950: 1,
  197550: 1,
  198855: 1,
  199655: 1,
  199958: 1,
  200614: 1,
  201111: 1,
  201414: 1,
  203418: 1,
  204218: 1,
  205725: 1,
  207123: 1,
  207628: 1,
  209935: 1,
  210011: 1,
  210415: 1,
  210617: 1,
  211417: 1,

### Frontal Middle Sulcus

In [59]:
def fms_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = (current_subject_hemi['S_front_sup']['inferior']).intersection(
                current_subject_hemi['Lat_Fis-ant-Horizont']['superior']).intersection(
                    current_subject_hemi['S_precentral-inf-part']['anterior'])
        sulci_found[current_subject]=sulcus_test

        
    return sulci_found

In [60]:
fms_check_left = fms_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
fms_check_right = fms_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [61]:
fms_prob_left = calculating_prob_of_sulcus_found(subject_ids, fms_check_left, 'Frontal_middle_sulcus', 'S_front_middle')
fms_prob_right = calculating_prob_of_sulcus_found(subject_ids, fms_check_right, 'Frontal_middle_sulcus', 'S_front_middle')

In [62]:
print(fms_prob_left, fms_prob_right)

(176, 572, 0.3076923076923077) (148, 572, 0.25874125874125875)


In [63]:
fms_left_accuracy = accuracy_check(subject_ids, fms_check_left, 'S_front_middle')
fms_right_accuracy = accuracy_check(subject_ids, fms_check_right, 'S_front_middle')

In [64]:
fms_left_accuracy

({102715: 3,
  103111: 3,
  105014: 2,
  105115: 2,
  105216: 2,
  105620: 2,
  106319: 1,
  106521: 3,
  108121: 2,
  108525: 3,
  108828: 1,
  109325: 3,
  111716: 3,
  112112: 2,
  115320: 3,
  116120: 1,
  116524: 3,
  117122: 2,
  118225: 3,
  121618: 1,
  122620: 1,
  124220: 1,
  125222: 2,
  125424: 1,
  125525: 3,
  127630: 3,
  131621: 3,
  131722: 3,
  131924: 2,
  132118: 3,
  134728: 2,
  135124: 2,
  135932: 3,
  136227: 2,
  142424: 2,
  145531: 2,
  145632: 4,
  150625: 1,
  150726: 4,
  151425: 3,
  152831: 3,
  153833: 4,
  154936: 1,
  155635: 3,
  157336: 3,
  158035: 4,
  158136: 2,
  158540: 1,
  158843: 1,
  159441: 3,
  161630: 2,
  161731: 2,
  162733: 1,
  163129: 3,
  167743: 2,
  171431: 3,
  173435: 3,
  173536: 2,
  176744: 2,
  177645: 3,
  177746: 4,
  178950: 1,
  179548: 3,
  180432: 4,
  181131: 1,
  181232: 1,
  182840: 3,
  185139: 1,
  187143: 3,
  187850: 4,
  188145: 2,
  189450: 3,
  192540: 3,
  195647: 3,
  196144: 3,
  196952: 3,
  197348: 3,

In [65]:
fms_right_accuracy

({102715: 1,
  103111: 2,
  105014: 2,
  105620: 1,
  106319: 1,
  106521: 2,
  109325: 1,
  111312: 1,
  113619: 2,
  113922: 2,
  114924: 2,
  117122: 1,
  118124: 1,
  121315: 1,
  122317: 2,
  122822: 1,
  125222: 2,
  127630: 2,
  127933: 2,
  130619: 1,
  131621: 1,
  132118: 1,
  133625: 1,
  135124: 1,
  135225: 2,
  135932: 3,
  142424: 1,
  143325: 2,
  145632: 1,
  146533: 2,
  148941: 2,
  150524: 2,
  150625: 3,
  151425: 2,
  151627: 3,
  153126: 3,
  153227: 3,
  154431: 3,
  154835: 4,
  155938: 2,
  165032: 4,
  167743: 1,
  169141: 1,
  169343: 3,
  169444: 1,
  169747: 1,
  172534: 3,
  172938: 1,
  173940: 1,
  175035: 2,
  176744: 2,
  177746: 3,
  178849: 1,
  180937: 2,
  181131: 1,
  185038: 1,
  195950: 1,
  196952: 2,
  198249: 3,
  199655: 1,
  201818: 1,
  206222: 1,
  208327: 2,
  209228: 1,
  212116: 3,
  214423: 2,
  217126: 2,
  220721: 3,
  224022: 1,
  239944: 1,
  245333: 3,
  250932: 2,
  255639: 1,
  285345: 2,
  285446: 3,
  286650: 2,
  304020: 3,

### Temporal Transverse Sulcus

In [66]:
def tts_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = (current_subject_hemi['Lat_Fis-post']['inferior']).intersection(
                current_subject_hemi['Lat_Fis-ant-Horizont']['posterior'])
        sulci_found[current_subject]=sulcus_test

        
    return sulci_found

In [67]:
tts_check_left = tts_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
tts_check_right = tts_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [68]:
tts_prob_left = calculating_prob_of_sulcus_found(subject_ids, tts_check_left, 'Temporal_transverse_sulcus', 'S_temporal_transverse')
tts_prob_right = calculating_prob_of_sulcus_found(subject_ids, tts_check_right, 'Temporal_transverse_sulcus', 'S_temporal_transverse')

In [69]:
print(tts_prob_left, tts_prob_right)

(197, 572, 0.34440559440559443) (238, 572, 0.4160839160839161)


In [70]:
tts_left_accuracy = accuracy_check(subject_ids, tts_check_left, 'S_temporal_transverse')
tts_right_accuracy = accuracy_check(subject_ids, tts_check_right, 'S_temporal_transverse')

In [71]:
tts_left_accuracy

({103515: 12,
  105115: 12,
  105620: 11,
  106521: 10,
  107321: 9,
  108828: 12,
  111009: 11,
  111413: 12,
  114419: 11,
  117122: 9,
  117930: 11,
  118124: 11,
  120717: 10,
  121416: 12,
  124220: 10,
  131419: 12,
  131621: 14,
  131722: 11,
  132118: 11,
  133019: 10,
  133625: 9,
  135932: 12,
  136227: 12,
  137027: 13,
  140420: 11,
  140925: 10,
  148133: 11,
  150524: 12,
  151223: 11,
  151627: 10,
  152427: 9,
  152831: 11,
  153429: 12,
  153833: 12,
  154532: 11,
  154734: 12,
  154835: 11,
  157437: 11,
  158035: 11,
  159239: 12,
  159441: 11,
  161327: 11,
  161630: 12,
  162026: 11,
  167440: 9,
  167743: 11,
  168341: 12,
  169141: 11,
  169343: 9,
  169444: 10,
  170934: 11,
  172029: 11,
  172534: 11,
  172938: 9,
  173435: 10,
  173940: 12,
  174437: 10,
  175742: 13,
  176037: 10,
  176744: 11,
  178142: 13,
  178748: 11,
  179245: 8,
  179548: 11,
  180432: 9,
  182032: 9,
  182840: 11,
  183034: 13,
  185442: 9,
  185947: 10,
  187547: 9,
  188145: 9,
  190

In [72]:
tts_right_accuracy

({103212: 13,
  103515: 10,
  104012: 14,
  104416: 9,
  104820: 10,
  105014: 12,
  105620: 10,
  107321: 12,
  108525: 13,
  109123: 12,
  111009: 11,
  111413: 10,
  112112: 13,
  112819: 10,
  113215: 10,
  113821: 11,
  114419: 8,
  115017: 11,
  115320: 12,
  116524: 12,
  117122: 12,
  118124: 12,
  121416: 11,
  122317: 9,
  123117: 13,
  124422: 12,
  125525: 10,
  127832: 11,
  130619: 10,
  131217: 11,
  132118: 11,
  133019: 12,
  133625: 13,
  133928: 11,
  134021: 11,
  135225: 11,
  136833: 10,
  137027: 12,
  137431: 9,
  138534: 8,
  140420: 9,
  140925: 9,
  143325: 13,
  145531: 12,
  150524: 8,
  150625: 11,
  150726: 11,
  151425: 13,
  151526: 13,
  153126: 13,
  153227: 12,
  153429: 12,
  153833: 12,
  154532: 9,
  154835: 9,
  156233: 10,
  156637: 11,
  159239: 9,
  159441: 10,
  162228: 11,
  162329: 8,
  163331: 13,
  164131: 11,
  165032: 12,
  167238: 9,
  167743: 11,
  168139: 11,
  168341: 9,
  169141: 9,
  169444: 10,
  170934: 10,
  171431: 10,
  17163

### Inferior Temporal Sulcus

In [73]:
def its_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = (current_subject_hemi['S_temporal_sup']['inferior']).intersection(
                current_subject_hemi['Lat_Fis-ant-Horizont']['posterior']).intersection(
                    current_subject_hemi['S_oc-temp_lat']['anterior']).intersection(
                        current_subject_hemi['S_collat_transv_ant']['superior'])
        sulci_found[current_subject]=sulcus_test

        
    return sulci_found

In [74]:
its_check_left = its_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
its_check_right = its_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [75]:
its_prob_left = calculating_prob_of_sulcus_found(subject_ids, its_check_left, 'Inferior_temporal_sulcus', 'S_temporal_inf')
its_prob_right = calculating_prob_of_sulcus_found(subject_ids, its_check_right, 'Inferior_temporal_sulcus', 'S_temporal_inf')

In [76]:
print(its_prob_left, its_prob_right)

(13, 572, 0.022727272727272728) (14, 572, 0.024475524475524476)


In [77]:
its_left_accuracy = accuracy_check(subject_ids, its_check_left, 'S_temporal_inf')
its_right_accuracy = accuracy_check(subject_ids, its_check_right, 'S_temporal_inf')

In [78]:
its_left_accuracy

({126931: 1,
  154431: 1,
  187850: 1,
  198855: 1,
  211316: 1,
  211417: 1,
  211720: 2,
  212419: 1,
  561242: 1,
  579665: 1,
  592455: 1,
  865363: 1,
  965367: 1},
 12,
 0.9230769230769231)

In [79]:
its_right_accuracy

({104416: 1,
  123925: 1,
  124422: 1,
  167440: 2,
  199958: 1,
  207628: 1,
  540436: 1,
  561242: 1,
  565452: 1,
  583858: 1,
  685058: 1,
  825048: 1,
  826454: 1,
  861456: 1},
 13,
 0.9285714285714286)

### Superior Temporal Sulcus

In [80]:
def sts_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = (current_subject_hemi['S_temporal_inf']['superior']).intersection(
                current_subject_hemi['S_postcentral']['inferior'])
        sulci_found[current_subject]=sulcus_test

        
    return sulci_found

In [81]:
sts_check_left = sts_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
sts_check_right = sts_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [82]:
sts_prob_left = calculating_prob_of_sulcus_found(subject_ids, sts_check_left, 'Superior_temporal_sulcus', 'S_temporal_sup')
sts_prob_right = calculating_prob_of_sulcus_found(subject_ids, sts_check_right, 'Superior_temporal_sulcus', 'S_temporal_sup')

In [83]:
print(sts_prob_left, sts_prob_right)

(375, 572, 0.6555944055944056) (402, 572, 0.7027972027972028)


In [84]:
sts_left_accuracy = accuracy_check(subject_ids, sts_check_left, 'S_temporal_sup')
sts_right_accuracy = accuracy_check(subject_ids, sts_check_right, 'S_temporal_sup')

In [85]:
sts_left_accuracy

({102715: 20,
  103111: 22,
  103212: 19,
  103515: 20,
  103818: 22,
  104012: 21,
  105014: 21,
  105115: 21,
  106016: 20,
  106521: 21,
  107422: 19,
  108121: 19,
  108525: 19,
  108828: 20,
  109123: 21,
  109325: 17,
  110411: 19,
  111312: 17,
  112112: 18,
  112819: 23,
  113215: 20,
  113619: 17,
  113821: 21,
  113922: 21,
  114924: 16,
  115017: 19,
  115320: 20,
  116120: 18,
  116524: 18,
  117122: 21,
  117324: 20,
  117930: 18,
  118225: 19,
  120717: 21,
  121315: 18,
  121416: 19,
  122620: 23,
  123117: 19,
  123420: 18,
  123925: 20,
  124220: 17,
  125222: 17,
  125424: 18,
  125525: 18,
  126931: 23,
  127630: 17,
  130922: 26,
  131419: 21,
  131621: 18,
  131823: 21,
  133625: 21,
  133827: 21,
  133928: 20,
  134021: 18,
  134324: 23,
  134728: 21,
  135124: 15,
  135225: 22,
  135932: 20,
  137027: 17,
  137431: 17,
  138534: 16,
  139839: 23,
  140420: 18,
  141826: 23,
  142424: 18,
  143325: 22,
  145531: 24,
  145632: 20,
  146533: 20,
  148133: 20,
  1489

In [86]:
sts_right_accuracy

({102715: 20,
  102816: 20,
  103414: 17,
  104012: 23,
  104416: 21,
  104820: 18,
  105014: 20,
  105216: 18,
  105620: 19,
  106016: 19,
  106521: 18,
  107018: 21,
  108121: 20,
  108525: 15,
  108828: 18,
  109123: 20,
  110411: 17,
  111312: 22,
  111716: 18,
  112112: 19,
  112819: 21,
  113215: 16,
  113619: 20,
  114924: 17,
  115017: 19,
  115320: 20,
  116120: 20,
  116524: 18,
  117324: 18,
  117930: 18,
  118124: 17,
  118225: 17,
  120717: 20,
  121416: 17,
  122317: 18,
  122822: 21,
  123420: 19,
  123925: 21,
  124220: 18,
  125222: 20,
  125424: 21,
  126931: 20,
  127630: 19,
  127832: 19,
  130619: 17,
  131217: 19,
  131419: 20,
  131621: 19,
  131722: 18,
  131823: 18,
  131924: 18,
  132118: 19,
  133019: 21,
  133625: 20,
  134021: 24,
  134324: 23,
  135124: 21,
  135225: 22,
  135528: 21,
  135932: 18,
  137027: 19,
  137431: 20,
  138534: 17,
  139435: 19,
  139839: 19,
  140925: 16,
  141826: 19,
  145531: 16,
  150524: 17,
  150625: 18,
  150726: 19,
  1514

### Subparietal Sulcus

In [87]:
def subparietal_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = current_subject_hemi['S_parieto_occipital']['superior'].intersection(
   current_subject_hemi['S_postcentral']['posterior']).intersection(
   current_subject_hemi['S_cingul-Marginalis']['inferior'])
        sulci_found[current_subject]=sulcus_test

    return sulci_found

In [88]:
subparietal_check_left = subparietal_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
subparietal_check_right = subparietal_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [89]:
subparietal_prob_left = calculating_prob_of_sulcus_found(subject_ids, subparietal_check_left, 'Subparietal_sulcus', 'S_subparietal')
subparietal_prob_right = calculating_prob_of_sulcus_found(subject_ids, subparietal_check_right, 'Subparietal_sulcus', 'S_subparietal')

In [90]:
print(subparietal_prob_left, subparietal_prob_right)

(37, 572, 0.06468531468531469) (37, 572, 0.06468531468531469)


In [91]:
subparietal_left_accuracy = accuracy_check(subject_ids, subparietal_check_left, 'S_subparietal')
subparietal_right_accuracy = accuracy_check(subject_ids, subparietal_check_right, 'S_subparietal')

In [92]:
subparietal_left_accuracy

({108121: 1,
  116120: 1,
  117324: 1,
  120717: 1,
  135225: 1,
  154835: 1,
  173536: 1,
  173738: 1,
  182840: 1,
  205220: 1,
  205725: 1,
  207628: 1,
  214726: 1,
  250932: 1,
  286650: 1,
  309636: 1,
  330324: 1,
  358144: 1,
  378756: 2,
  393550: 1,
  421226: 1,
  441939: 1,
  485757: 1,
  566454: 1,
  581349: 1,
  588565: 1,
  598568: 1,
  650746: 1,
  715950: 1,
  789373: 1,
  826353: 1,
  841349: 1,
  877168: 1,
  889579: 1,
  891667: 1,
  894774: 1,
  896879: 1},
 36,
 0.972972972972973)

In [93]:
subparietal_right_accuracy

({103414: 1,
  103818: 1,
  109325: 1,
  116120: 1,
  118124: 1,
  120717: 1,
  124826: 1,
  132118: 1,
  150625: 1,
  155231: 1,
  165032: 3,
  169444: 1,
  171633: 1,
  190031: 1,
  200311: 2,
  211316: 1,
  236130: 1,
  280739: 1,
  281135: 1,
  285345: 1,
  356948: 1,
  366446: 1,
  385046: 1,
  441939: 1,
  485757: 1,
  581349: 1,
  664757: 1,
  742549: 1,
  761957: 1,
  773257: 1,
  779370: 1,
  789373: 1,
  792766: 1,
  814649: 1,
  816653: 1,
  835657: 1,
  957974: 1},
 35,
 0.9459459459459459)

### Superior Frontal Sulcus

In [94]:
def sfs_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = ((current_subject_hemi["S_precentral-sup-part"]['anterior']).union(
            current_subject_hemi["S_precentral-inf-part"]['anterior'])).intersection(
            current_subject_hemi["S_front_inf"]['superior'])
        
        sulci_found[current_subject]=sulcus_test

    return sulci_found

In [95]:
SFS_check_left = sfs_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
SFS_check_right = sfs_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [96]:
SFS_prob_left = calculating_prob_of_sulcus_found(subject_ids, SFS_check_left, 'Superior_frontal_sulcus', 'S_front_sup')
SFS_prob_right = calculating_prob_of_sulcus_found(subject_ids, SFS_check_right, 'Superior_frontal_sulcus', 'S_front_sup')

In [97]:
print(SFS_prob_left, SFS_prob_right)

(225, 572, 0.39335664335664333) (112, 572, 0.1958041958041958)


In [98]:
SFS_left_accuracy = accuracy_check(subject_ids, SFS_check_left, 'S_front_sup')
SFS_right_accuracy = accuracy_check(subject_ids, SFS_check_right, 'S_front_sup')

In [99]:
SFS_left_accuracy

({103212: 1,
  103414: 1,
  103818: 1,
  104416: 1,
  104820: 1,
  105014: 1,
  105115: 1,
  106016: 1,
  106521: 1,
  107018: 1,
  107321: 1,
  108525: 1,
  108828: 1,
  110411: 1,
  111312: 1,
  113215: 1,
  114419: 1,
  114924: 2,
  118124: 1,
  121618: 1,
  122317: 1,
  123117: 1,
  124826: 1,
  125424: 2,
  127832: 1,
  127933: 1,
  131823: 1,
  133625: 1,
  134021: 1,
  135528: 1,
  136227: 1,
  136833: 1,
  145531: 1,
  145632: 1,
  146533: 1,
  148133: 1,
  151425: 1,
  151526: 1,
  151930: 1,
  152427: 1,
  152831: 1,
  153025: 1,
  153429: 1,
  153833: 1,
  154431: 1,
  154532: 1,
  154734: 1,
  154835: 1,
  155231: 1,
  155938: 1,
  157336: 1,
  158136: 1,
  161327: 1,
  162026: 1,
  162329: 1,
  163129: 1,
  165032: 1,
  167238: 1,
  167743: 1,
  169141: 1,
  171431: 1,
  172332: 1,
  173940: 1,
  175237: 1,
  176037: 1,
  176744: 1,
  177645: 1,
  177746: 1,
  180129: 1,
  180432: 1,
  180836: 1,
  181131: 1,
  181232: 1,
  182840: 1,
  185947: 1,
  186141: 1,
  188145: 1,

In [100]:
SFS_right_accuracy

({104012: 1,
  108121: 1,
  111312: 1,
  113619: 2,
  115017: 1,
  121618: 1,
  127630: 1,
  131419: 1,
  131621: 1,
  131924: 1,
  134021: 1,
  135225: 1,
  135528: 1,
  142424: 1,
  143325: 1,
  144226: 1,
  148133: 1,
  151526: 1,
  151930: 1,
  153126: 1,
  154835: 1,
  154936: 1,
  156233: 1,
  158843: 1,
  162026: 1,
  167743: 1,
  168947: 1,
  169343: 1,
  171431: 1,
  172534: 1,
  175237: 1,
  175439: 1,
  177746: 1,
  178849: 2,
  180129: 1,
  191942: 2,
  192843: 1,
  197449: 1,
  198249: 2,
  199150: 1,
  199251: 1,
  199655: 3,
  207123: 1,
  207426: 1,
  209834: 1,
  209935: 1,
  210617: 1,
  211114: 1,
  211922: 1,
  212116: 1,
  214019: 1,
  214726: 1,
  224022: 1,
  251833: 1,
  268850: 1,
  285446: 1,
  304020: 1,
  329440: 1,
  329844: 1,
  333330: 1,
  334635: 1,
  349244: 1,
  352132: 1,
  352738: 1,
  355542: 1,
  356948: 1,
  368551: 1,
  378756: 1,
  390645: 1,
  397760: 1,
  414229: 1,
  422632: 1,
  424939: 1,
  436845: 1,
  461743: 1,
  465852: 1,
  485757: 1,

### Calcarine Sulcus

In [101]:
All_subject_and_hemisphere_sets[144226]['left']['S_calcarine']

{'anterior': {'Lat_Fis-ant-Horizont',
  'Lat_Fis-ant-Vertical',
  'S_central',
  'S_circular_insula_ant',
  'S_circular_insula_inf',
  'S_circular_insula_sup',
  'S_collat_transv_ant',
  'S_collat_transv_post',
  'S_front_inf',
  'S_front_middle',
  'S_front_sup',
  'S_interm_prim-Jensen',
  'S_intrapariet_and_P_trans',
  'S_occipital_ant',
  'S_orbital-H_Shaped',
  'S_orbital_lateral',
  'S_orbital_med-olfact',
  'S_postcentral',
  'S_precentral-inf-part',
  'S_precentral-sup-part',
  'S_suborbital',
  'S_temporal_sup',
  'S_temporal_transverse'},
 'inferior': {'Lat_Fis-ant-Horizont',
  'S_circular_insula_ant',
  'S_collat_transv_ant',
  'S_oc-temp_med_and_Lingual',
  'S_orbital-H_Shaped',
  'S_orbital_med-olfact',
  'S_suborbital'},
 'posterior': set(),
 'superior': {'Lat_Fis-ant-Vertical',
  'Lat_Fis-post',
  'S_central',
  'S_cingul-Marginalis',
  'S_circular_insula_sup',
  'S_front_inf',
  'S_front_middle',
  'S_front_sup',
  'S_interm_prim-Jensen',
  'S_intrapariet_and_P_trans',


In [102]:
def calcarine_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = (current_subject_hemi['S_parieto_occipital']['inferior'].intersection(
            current_subject_hemi['S_oc_middle_and_Lunatus']['anterior']).intersection(
                 current_subject_hemi['Lat_Fis-post']['posterior']).intersection(
                    current_subject_hemi['S_collat_transv_ant']['superior']))
        
        sulci_found[current_subject]=sulcus_test

    return sulci_found

In [103]:
Calcarine_check_left = calcarine_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
Calcarine_check_right = calcarine_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [104]:
Calcarine_prob_left = calculating_prob_of_sulcus_found(subject_ids, Calcarine_check_left, 'Calcarine_sulcus', 'S_calcarine')
Calcarine_prob_right = calculating_prob_of_sulcus_found(subject_ids, Calcarine_check_right, 'Calcarine_sulcus', 'S_calcarine')

In [105]:
print(Calcarine_prob_left, Calcarine_prob_right)

(73, 572, 0.12762237762237763) (63, 572, 0.11013986013986014)


In [106]:
Calcarine_left_accuracy = accuracy_check(subject_ids, Calcarine_check_left, 'S_calcarine')
Calcarine_right_accuracy = accuracy_check(subject_ids, Calcarine_check_right, 'S_calcarine')

In [107]:
Calcarine_left_accuracy

({103515: 4,
  106016: 4,
  108525: 5,
  113821: 4,
  114924: 3,
  123117: 5,
  123420: 3,
  124826: 4,
  125222: 5,
  131419: 5,
  134324: 2,
  146533: 7,
  148941: 6,
  150524: 5,
  151627: 6,
  152831: 2,
  154734: 3,
  158843: 4,
  168341: 4,
  169343: 5,
  171633: 4,
  172130: 5,
  173536: 4,
  176037: 4,
  181131: 5,
  182032: 4,
  182840: 4,
  185947: 3,
  200109: 5,
  201414: 4,
  204218: 3,
  205826: 4,
  209228: 4,
  212217: 4,
  212419: 3,
  233326: 5,
  251833: 5,
  281135: 5,
  336841: 3,
  351938: 5,
  355542: 5,
  356948: 5,
  371843: 5,
  377451: 4,
  389357: 5,
  441939: 4,
  459453: 4,
  485757: 5,
  499566: 5,
  522434: 2,
  536647: 3,
  541943: 6,
  573249: 4,
  587664: 2,
  592455: 5,
  673455: 3,
  679770: 3,
  685058: 4,
  702133: 3,
  705341: 6,
  742549: 5,
  774663: 3,
  841349: 3,
  843151: 3,
  870861: 5,
  872158: 5,
  917558: 5,
  922854: 1,
  930449: 5,
  957974: 5,
  958976: 3,
  965367: 5,
  994273: 4},
 1,
 0.0136986301369863)

In [108]:
Calcarine_right_accuracy

({108525: 4,
  108828: 4,
  113215: 3,
  114419: 1,
  114924: 2,
  115017: 3,
  118124: 3,
  124826: 2,
  130619: 4,
  131419: 3,
  131924: 3,
  134728: 2,
  136227: 4,
  138534: 5,
  150524: 5,
  151627: 3,
  152831: 2,
  158540: 3,
  167743: 5,
  168341: 2,
  169444: 3,
  171633: 4,
  172130: 3,
  172534: 5,
  173738: 4,
  181131: 6,
  185139: 5,
  192540: 4,
  197550: 4,
  200614: 7,
  203418: 4,
  212217: 2,
  224022: 3,
  255639: 5,
  339847: 4,
  346137: 2,
  355542: 4,
  397760: 4,
  436845: 3,
  522434: 1,
  536647: 3,
  566454: 5,
  573249: 3,
  573451: 3,
  583858: 5,
  592455: 3,
  628248: 4,
  644044: 5,
  679770: 4,
  685058: 3,
  729557: 6,
  745555: 4,
  748662: 4,
  751550: 3,
  774663: 3,
  782157: 3,
  835657: 6,
  861456: 4,
  889579: 1,
  898176: 4,
  899885: 4,
  930449: 5,
  958976: 2},
 3,
 0.047619047619047616)

### Postcentral Sulcus

In [109]:
All_subject_and_hemisphere_sets[826454]['left']['S_postcentral']

{'anterior': {'Lat_Fis-ant-Horizont',
  'Lat_Fis-ant-Vertical',
  'S_central',
  'S_circular_insula_ant',
  'S_circular_insula_sup',
  'S_front_inf',
  'S_front_middle',
  'S_front_sup',
  'S_orbital-H_Shaped',
  'S_orbital_lateral',
  'S_orbital_med-olfact',
  'S_precentral-inf-part',
  'S_precentral-sup-part',
  'S_suborbital'},
 'inferior': {'Lat_Fis-ant-Horizont',
  'Lat_Fis-ant-Vertical',
  'Lat_Fis-post',
  'S_calcarine',
  'S_cingul-Marginalis',
  'S_circular_insula_ant',
  'S_circular_insula_inf',
  'S_circular_insula_sup',
  'S_collat_transv_ant',
  'S_collat_transv_post',
  'S_front_inf',
  'S_front_middle',
  'S_interm_prim-Jensen',
  'S_oc-temp_lat',
  'S_oc-temp_med_and_Lingual',
  'S_oc_middle_and_Lunatus',
  'S_oc_sup_and_transversal',
  'S_occipital_ant',
  'S_orbital-H_Shaped',
  'S_orbital_lateral',
  'S_orbital_med-olfact',
  'S_parieto_occipital',
  'S_pericallosal',
  'S_precentral-inf-part',
  'S_suborbital',
  'S_subparietal',
  'S_temporal_inf',
  'S_temporal_su

In [110]:
def postcentral_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = (current_subject_hemi['S_occipital_ant']['anterior']).intersection((
            current_subject_hemi['Lat_Fis-ant-Horizont']['superior'].union(
               current_subject_hemi['Lat_Fis-ant-Vertical']['superior'].union(
                  current_subject_hemi['Lat_Fis-post']['superior']))).intersection(
                       current_subject_hemi['S_central']['posterior']))
        
        sulci_found[current_subject]=sulcus_test

    return sulci_found

In [111]:
Postcentral_check_left = postcentral_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
Postcentral_check_right = postcentral_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [112]:
Postcentral_prob_left = calculating_prob_of_sulcus_found(subject_ids, Postcentral_check_left, 'Postcentral_sulcus', 'S_postcentral')
Postcentral_prob_right = calculating_prob_of_sulcus_found(subject_ids, Postcentral_check_right, 'Postcentral_sulcus', 'S_postcentral')

In [113]:
print(Postcentral_prob_left, Postcentral_prob_right)

(413, 572, 0.722027972027972) (340, 572, 0.5944055944055944)


In [114]:
PoCS_left_accuracy = accuracy_check(subject_ids, Postcentral_check_left, 'S_postcentral')
PoCS_right_accuracy = accuracy_check(subject_ids, Postcentral_check_right, 'S_postcentral')

In [115]:
PoCS_left_accuracy

({102816: 5,
  103111: 6,
  103414: 5,
  103818: 5,
  104416: 5,
  104820: 4,
  105115: 5,
  105216: 5,
  105620: 4,
  106319: 6,
  106521: 2,
  107018: 4,
  107321: 7,
  108121: 6,
  108525: 6,
  109123: 3,
  109325: 6,
  110411: 3,
  111009: 5,
  111312: 6,
  111716: 7,
  112112: 4,
  112819: 5,
  113215: 4,
  113619: 5,
  113922: 5,
  114419: 5,
  114924: 1,
  115320: 4,
  116120: 5,
  116524: 4,
  117930: 4,
  118124: 6,
  118225: 3,
  120717: 7,
  121315: 7,
  121416: 5,
  122317: 4,
  122620: 6,
  122822: 6,
  123117: 3,
  123420: 5,
  123925: 5,
  124422: 7,
  124826: 6,
  125424: 5,
  125525: 7,
  126931: 5,
  127630: 4,
  127832: 5,
  127933: 6,
  130619: 6,
  130922: 4,
  131217: 3,
  131419: 4,
  131621: 3,
  131722: 7,
  131823: 4,
  133625: 3,
  133827: 4,
  133928: 5,
  134021: 6,
  135124: 5,
  136227: 5,
  136833: 4,
  137027: 6,
  137431: 5,
  138534: 5,
  139435: 6,
  139839: 4,
  140420: 4,
  140925: 6,
  141826: 5,
  143325: 5,
  144226: 5,
  145632: 7,
  146533: 6,

In [116]:
PoCS_right_accuracy

({103111: 4,
  103212: 4,
  103414: 5,
  103515: 3,
  103818: 3,
  105014: 6,
  105115: 7,
  105620: 4,
  106319: 5,
  106521: 4,
  107018: 3,
  107321: 4,
  108121: 5,
  108323: 2,
  108525: 4,
  109123: 3,
  109325: 5,
  111716: 6,
  112819: 4,
  113215: 5,
  113619: 6,
  113821: 5,
  114419: 3,
  116524: 4,
  117930: 2,
  118124: 5,
  120717: 5,
  121315: 4,
  121618: 4,
  123420: 5,
  123925: 5,
  124826: 5,
  125424: 5,
  125525: 3,
  126931: 6,
  127630: 4,
  127832: 6,
  127933: 5,
  130619: 3,
  131419: 5,
  131722: 6,
  131823: 3,
  132118: 3,
  133019: 3,
  133827: 4,
  133928: 2,
  135124: 5,
  135528: 5,
  135932: 5,
  136833: 4,
  137027: 4,
  137431: 4,
  138534: 3,
  139435: 3,
  140925: 5,
  141826: 4,
  142424: 4,
  143325: 3,
  144226: 2,
  145632: 3,
  146533: 5,
  148133: 5,
  148941: 3,
  150524: 6,
  151223: 3,
  151425: 6,
  152225: 3,
  153025: 4,
  153126: 5,
  154431: 6,
  154835: 4,
  154936: 3,
  155231: 6,
  155635: 4,
  155938: 4,
  156031: 4,
  156233: 6,

### Inferior Frontal Sulcus

In [117]:
def ifs_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = current_subject_hemi['S_front_sup']['inferior'].intersection(
           current_subject_hemi['S_precentral-sup-part']['anterior'].union(
                current_subject_hemi['S_precentral-inf-part']['anterior']))
#                 current_subject_hemi["Lat_Fis-ant-Horizont"]['superior'].union(
#                     current_subject_hemi["Lat_Fis-ant-Vertical"]['superior'].union(
#                         current_subject_hemi["Lat_Fis-post"]['superior'])))

        sulci_found[current_subject]=sulcus_test

    return sulci_found

In [118]:
IFS_check_left = ifs_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
IFS_check_right = ifs_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [119]:
IFS_prob_left = calculating_prob_of_sulcus_found(subject_ids, IFS_check_left, 'Inferior_frontal_sulcus', 'S_front_inf')
IFS_prob_right = calculating_prob_of_sulcus_found(subject_ids, IFS_check_right, 'Inferior_frontal_sulcus', 'S_front_inf')

In [120]:
print(IFS_prob_left, IFS_prob_right)

(462, 572, 0.8076923076923077) (494, 572, 0.8636363636363636)


In [121]:
IFS_left_accuracy = accuracy_check(subject_ids, IFS_check_left, 'S_front_inf')
IFS_right_accuracy = accuracy_check(subject_ids, IFS_check_right, 'S_front_inf')

In [122]:
IFS_left_accuracy

({102715: 11,
  103111: 10,
  103212: 8,
  103414: 9,
  103515: 8,
  103818: 9,
  104416: 9,
  104820: 8,
  105014: 8,
  105115: 8,
  105216: 10,
  105620: 10,
  106016: 9,
  106319: 8,
  106521: 10,
  107018: 9,
  107321: 8,
  107422: 8,
  108121: 10,
  108323: 9,
  108525: 9,
  108828: 10,
  109325: 9,
  110411: 9,
  111312: 9,
  111413: 8,
  111716: 9,
  112112: 10,
  112819: 7,
  113215: 9,
  113619: 8,
  113821: 9,
  114419: 9,
  114924: 9,
  115017: 9,
  115320: 10,
  116120: 8,
  116524: 9,
  117122: 8,
  117324: 8,
  117930: 10,
  118124: 8,
  120717: 8,
  121416: 8,
  121618: 9,
  122317: 8,
  122822: 8,
  123117: 9,
  123925: 7,
  124220: 10,
  124422: 8,
  124826: 8,
  125222: 9,
  125424: 9,
  125525: 9,
  126931: 8,
  127630: 8,
  127832: 8,
  130619: 9,
  131217: 7,
  131419: 9,
  131621: 9,
  131722: 9,
  131823: 7,
  131924: 9,
  132118: 10,
  133019: 8,
  133625: 9,
  133827: 9,
  133928: 9,
  134021: 8,
  134728: 9,
  135225: 7,
  135528: 9,
  135932: 9,
  136227: 9,


In [123]:
IFS_right_accuracy

({102715: 10,
  102816: 8,
  103111: 9,
  103212: 8,
  103414: 9,
  103515: 8,
  104012: 9,
  104416: 9,
  104820: 8,
  105014: 9,
  105115: 8,
  105216: 7,
  105620: 10,
  106319: 10,
  106521: 10,
  107321: 8,
  107422: 8,
  108121: 8,
  108323: 7,
  108525: 8,
  108828: 10,
  109123: 9,
  109325: 9,
  110411: 10,
  111009: 8,
  111312: 8,
  111413: 9,
  112112: 8,
  112819: 8,
  113215: 8,
  113619: 10,
  113821: 7,
  114419: 10,
  114924: 9,
  115017: 8,
  115320: 9,
  117930: 9,
  118124: 10,
  118225: 9,
  120717: 9,
  121315: 9,
  121416: 8,
  121618: 8,
  122317: 10,
  122822: 9,
  123117: 9,
  123420: 8,
  123925: 8,
  125222: 9,
  125525: 8,
  126931: 9,
  127630: 10,
  127832: 8,
  127933: 9,
  130922: 8,
  131419: 8,
  131621: 10,
  131823: 8,
  131924: 8,
  132118: 10,
  133019: 8,
  133625: 10,
  133827: 9,
  134021: 7,
  134324: 8,
  134728: 9,
  135124: 9,
  135225: 9,
  135528: 9,
  135932: 10,
  136227: 9,
  136833: 10,
  137027: 8,
  137431: 9,
  138534: 8,
  139435:

### Pericallosal Sulcus

In [124]:
All_subject_and_hemisphere_sets[958976]['left']['S_pericallosal']

{'anterior': {'Lat_Fis-ant-Horizont',
  'Lat_Fis-ant-Vertical',
  'S_collat_transv_ant',
  'S_front_middle',
  'S_orbital-H_Shaped',
  'S_orbital_lateral'},
 'inferior': {'S_collat_transv_ant',
  'S_oc-temp_lat',
  'S_orbital-H_Shaped',
  'S_orbital_med-olfact',
  'S_suborbital',
  'S_temporal_inf'},
 'posterior': {'Lat_Fis-post',
  'S_circular_insula_inf',
  'S_collat_transv_post',
  'S_intrapariet_and_P_trans',
  'S_oc_middle_and_Lunatus',
  'S_oc_sup_and_transversal',
  'S_occipital_ant',
  'S_parieto_occipital',
  'S_subparietal',
  'S_temporal_transverse'},
 'superior': {'Lat_Fis-ant-Vertical',
  'Lat_Fis-post',
  'S_central',
  'S_cingul-Marginalis',
  'S_circular_insula_sup',
  'S_front_inf',
  'S_front_middle',
  'S_front_sup',
  'S_interm_prim-Jensen',
  'S_intrapariet_and_P_trans',
  'S_oc_sup_and_transversal',
  'S_parieto_occipital',
  'S_postcentral',
  'S_precentral-sup-part',
  'S_subparietal'}}

In [125]:
def pericallosal_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = current_subject_hemi['S_cingul-Marginalis']['inferior'].intersection(
                current_subject_hemi['S_occipital_ant']['anterior']).intersection(
                        current_subject_hemi['S_oc-temp_lat']['superior']).intersection(
                            current_subject_hemi['Lat_Fis-ant-Vertical']['posterior'])

        sulci_found[current_subject]=sulcus_test

    return sulci_found

In [126]:
Pericallosal_check_left = pericallosal_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
Pericallosal_check_right = pericallosal_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [127]:
Pericallosal_prob_left = calculating_prob_of_sulcus_found(subject_ids, Pericallosal_check_left, 'Pericallosal_sulcus', 'S_pericallosal')
Pericallosal_prob_right = calculating_prob_of_sulcus_found(subject_ids, Pericallosal_check_right, 'Pericallosal_sulcus', 'S_pericallosal')

In [128]:
print(Pericallosal_prob_left, Pericallosal_prob_right)

(16, 572, 0.027972027972027972) (1, 572, 0.0017482517482517483)


In [129]:
Pericallosal_left_accuracy = accuracy_check(subject_ids, Pericallosal_check_left, 'S_pericallosal')
Pericallosal_right_accuracy = accuracy_check(subject_ids, Pericallosal_check_right, 'S_pericallosal')

In [130]:
Pericallosal_left_accuracy

({117122: 9,
  117930: 4,
  124220: 7,
  145531: 9,
  145632: 7,
  150524: 8,
  152427: 7,
  201111: 8,
  207426: 6,
  209935: 6,
  217126: 8,
  281135: 6,
  285345: 8,
  486759: 4,
  763557: 8,
  958976: 7},
 0,
 0.0)

In [131]:
Pericallosal_right_accuracy

({693764: 6}, 0, 0.0)

### Lateral Occipit(o-Temporal) Sulcus

In [132]:
All_subject_and_hemisphere_sets[693764]['left']['S_oc-temp_lat']

{'anterior': {'Lat_Fis-ant-Horizont',
  'Lat_Fis-ant-Vertical',
  'Lat_Fis-post',
  'S_central',
  'S_cingul-Marginalis',
  'S_circular_insula_ant',
  'S_circular_insula_inf',
  'S_circular_insula_sup',
  'S_collat_transv_ant',
  'S_front_inf',
  'S_front_middle',
  'S_front_sup',
  'S_interm_prim-Jensen',
  'S_intrapariet_and_P_trans',
  'S_orbital-H_Shaped',
  'S_orbital_lateral',
  'S_orbital_med-olfact',
  'S_postcentral',
  'S_precentral-inf-part',
  'S_precentral-sup-part',
  'S_suborbital',
  'S_temporal_inf',
  'S_temporal_sup',
  'S_temporal_transverse'},
 'inferior': {'S_collat_transv_ant',
  'S_orbital-H_Shaped',
  'S_orbital_med-olfact'},
 'posterior': {'S_collat_transv_post',
  'S_oc_middle_and_Lunatus',
  'S_parieto_occipital'},
 'superior': {'Lat_Fis-ant-Vertical',
  'Lat_Fis-post',
  'S_calcarine',
  'S_central',
  'S_cingul-Marginalis',
  'S_circular_insula_sup',
  'S_front_inf',
  'S_front_middle',
  'S_front_sup',
  'S_interm_prim-Jensen',
  'S_intrapariet_and_P_tran

In [133]:
def los_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = current_subject_hemi['S_parieto_occipital']['inferior'].intersection(
            current_subject_hemi['Lat_Fis-post']['posterior']).intersection(
               current_subject_hemi['S_oc_middle_and_Lunatus']['anterior'])

        sulci_found[current_subject]=sulcus_test

    return sulci_found

In [134]:
LOS_check_left = los_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
LOS_check_right = los_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [135]:
LOS_prob_left = calculating_prob_of_sulcus_found(subject_ids, LOS_check_left, 'Lateral_occipito-temporal_sulcus', 'S_oc-temp_lat')
LOS_prob_right = calculating_prob_of_sulcus_found(subject_ids, LOS_check_right, 'Lateral_occipito-temporal_sulcus', 'S_oc-temp_lat')

In [136]:
print(LOS_prob_left, LOS_prob_right)

(378, 572, 0.6608391608391608) (388, 572, 0.6783216783216783)


In [137]:
LOS_left_accuracy = accuracy_check(subject_ids, LOS_check_left, 'S_oc-temp_lat')
LOS_right_accuracy = accuracy_check(subject_ids, LOS_check_right, 'S_oc-temp_lat')

In [138]:
LOS_left_accuracy

({103212: 3,
  103414: 4,
  104012: 4,
  104416: 3,
  104820: 6,
  105014: 3,
  105216: 5,
  105620: 3,
  106016: 5,
  106319: 6,
  107018: 5,
  107321: 4,
  107422: 3,
  108121: 5,
  108525: 5,
  109123: 5,
  109325: 3,
  110411: 4,
  111009: 5,
  111716: 5,
  112112: 5,
  112819: 4,
  113619: 4,
  113821: 6,
  113922: 6,
  115017: 4,
  116120: 4,
  117122: 4,
  117930: 4,
  118124: 5,
  121315: 6,
  122317: 4,
  122620: 3,
  122822: 3,
  123117: 5,
  124422: 6,
  124826: 5,
  125222: 5,
  125424: 5,
  125525: 4,
  127630: 4,
  127832: 4,
  130619: 4,
  130922: 4,
  131217: 4,
  131419: 5,
  131722: 4,
  131924: 3,
  132118: 5,
  133827: 3,
  134021: 4,
  134728: 5,
  135225: 3,
  135932: 5,
  136833: 5,
  137431: 4,
  138534: 4,
  139435: 5,
  139839: 5,
  140420: 4,
  140925: 4,
  141826: 5,
  146533: 7,
  148133: 3,
  150524: 5,
  150625: 3,
  151425: 5,
  151526: 5,
  151627: 7,
  151930: 4,
  152225: 4,
  152427: 1,
  153126: 5,
  153227: 4,
  153429: 4,
  153833: 4,
  154734: 4,

In [139]:
LOS_right_accuracy

({102715: 4,
  102816: 5,
  103212: 3,
  103414: 4,
  103818: 3,
  104012: 5,
  104416: 3,
  104820: 4,
  105115: 5,
  105216: 4,
  105620: 5,
  106319: 3,
  107018: 4,
  107321: 3,
  108121: 5,
  108323: 4,
  108828: 4,
  109325: 3,
  110411: 5,
  111009: 5,
  111312: 5,
  111716: 4,
  112112: 7,
  112819: 4,
  113821: 4,
  113922: 2,
  114924: 4,
  115017: 4,
  116120: 2,
  117122: 4,
  117324: 3,
  117930: 4,
  118124: 5,
  118225: 3,
  120717: 4,
  121315: 5,
  122317: 2,
  122620: 4,
  122822: 3,
  123925: 4,
  124220: 1,
  124422: 4,
  125222: 4,
  125424: 6,
  125525: 5,
  126931: 5,
  127630: 5,
  127832: 4,
  127933: 2,
  130619: 4,
  130922: 5,
  131217: 3,
  131419: 4,
  131722: 4,
  131823: 5,
  132118: 3,
  133827: 3,
  134021: 5,
  134324: 3,
  135124: 4,
  135225: 5,
  135528: 3,
  135932: 4,
  136833: 3,
  137431: 2,
  138534: 5,
  139435: 3,
  139839: 5,
  140420: 4,
  141826: 3,
  143325: 3,
  144226: 5,
  146533: 4,
  148133: 3,
  148941: 3,
  150524: 5,
  151627: 5,

### Suborbital Sulcus

In [140]:
def suborbital_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = current_subject_hemi['S_pericallosal']['anterior'].intersection(
           current_subject_hemi['S_orbital_med-olfact']['superior'])#.intersection(
#                current_subject_hemi['Lat_Fis-ant-Vertical']['superior']).intersection(
#                     current_subject_hemi['S_oc-temp_lat']['anterior'])

        sulci_found[current_subject]=sulcus_test

    return sulci_found

In [141]:
suborbital_check_left = suborbital_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
suborbital_check_right = suborbital_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [142]:
suborbital_prob_left = calculating_prob_of_sulcus_found(subject_ids, suborbital_check_left, 'Suborbital_sulcus', 'S_suborbital')
suborbital_prob_right = calculating_prob_of_sulcus_found(subject_ids, suborbital_check_right, 'Suborbital_sulcus', 'S_suborbital')

In [143]:
print(suborbital_prob_left, suborbital_prob_right)

(74, 572, 0.12937062937062938) (281, 572, 0.49125874125874125)


In [144]:
suborbital_left_accuracy = accuracy_check(subject_ids, suborbital_check_left, 'S_suborbital')
suborbital_right_accuracy = accuracy_check(subject_ids, suborbital_check_right, 'S_suborbital')

In [145]:
suborbital_left_accuracy

({105620: 5,
  107422: 4,
  112112: 5,
  114924: 3,
  115017: 3,
  117930: 5,
  125525: 4,
  126931: 5,
  137431: 4,
  145531: 6,
  145632: 6,
  151223: 4,
  156031: 5,
  156637: 4,
  161327: 4,
  162329: 4,
  172938: 6,
  173435: 6,
  176037: 4,
  178748: 5,
  179548: 2,
  180129: 4,
  185442: 4,
  187547: 4,
  189450: 3,
  192843: 6,
  195041: 5,
  195849: 6,
  196144: 5,
  199958: 5,
  204521: 5,
  205725: 6,
  206222: 7,
  206525: 5,
  208024: 4,
  208226: 3,
  209329: 6,
  211114: 5,
  211215: 4,
  211316: 4,
  274542: 6,
  386250: 4,
  397861: 4,
  413934: 5,
  448347: 4,
  453441: 3,
  459453: 2,
  465852: 4,
  467351: 4,
  469961: 4,
  486759: 4,
  522434: 5,
  529953: 5,
  561242: 4,
  573249: 4,
  583858: 6,
  599671: 6,
  627549: 5,
  654754: 4,
  680452: 5,
  751348: 5,
  759869: 5,
  789373: 6,
  802844: 3,
  809252: 7,
  820745: 4,
  833249: 5,
  843151: 4,
  845458: 4,
  856766: 4,
  861456: 5,
  901038: 5,
  910241: 5,
  922854: 2},
 0,
 0.0)

In [146]:
suborbital_right_accuracy

({103212: 3,
  103414: 4,
  103818: 5,
  104012: 5,
  104416: 3,
  104820: 4,
  106016: 4,
  106319: 4,
  106521: 3,
  107422: 3,
  108828: 5,
  110411: 5,
  111312: 3,
  112819: 4,
  113215: 3,
  113619: 5,
  113922: 4,
  115017: 4,
  117122: 4,
  118124: 4,
  121416: 4,
  122317: 5,
  122620: 2,
  122822: 3,
  123117: 3,
  124422: 4,
  125222: 5,
  125424: 4,
  130619: 2,
  130922: 3,
  131217: 4,
  131419: 5,
  131722: 3,
  131823: 3,
  132118: 3,
  133019: 5,
  133625: 4,
  133827: 4,
  134021: 4,
  134728: 4,
  135225: 4,
  135528: 4,
  135932: 4,
  136227: 5,
  137431: 3,
  138534: 5,
  139839: 6,
  140420: 3,
  141826: 5,
  143325: 6,
  151223: 3,
  151425: 4,
  151526: 6,
  151627: 4,
  151930: 5,
  152831: 4,
  153126: 2,
  153227: 3,
  153429: 4,
  154734: 3,
  156031: 3,
  156637: 4,
  157336: 4,
  158035: 4,
  158540: 4,
  158843: 4,
  159239: 4,
  159441: 3,
  161731: 2,
  162026: 3,
  162329: 1,
  162733: 5,
  163129: 3,
  167238: 4,
  168341: 2,
  168947: 3,
  169141: 4,

### Sulcus Intermedius Primus of Jensen

In [147]:
def interm_jen_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = current_subject_hemi['S_intrapariet_and_P_trans']['inferior'].intersection(
           current_subject_hemi['S_postcentral']['posterior']).intersection(
               current_subject_hemi['Lat_Fis-ant-Vertical']['superior']).intersection(
                    current_subject_hemi['S_oc-temp_lat']['anterior'])

        sulci_found[current_subject]=sulcus_test

    return sulci_found

In [148]:
interm_jen_check_left = interm_jen_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
interm_jen_check_right = interm_jen_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [149]:
interm_jen_prob_left = calculating_prob_of_sulcus_found(subject_ids, interm_jen_check_left, 'Sulcus_intermedius_primus_of_jensen', 'S_interm_prim-Jensen')
interm_jen_prob_right = calculating_prob_of_sulcus_found(subject_ids, interm_jen_check_right, 'Sulcus_intermedius_primus_of_jensen', 'S_interm_prim-Jensen')

In [150]:
print(interm_jen_prob_left, interm_jen_prob_right)

(278, 572, 0.486013986013986) (9, 572, 0.015734265734265736)


In [151]:
interm_jen_left_accuracy = accuracy_check(subject_ids, interm_jen_check_left, 'S_interm_prim-Jensen')
interm_jen_right_accuracy = accuracy_check(subject_ids, interm_jen_check_right, 'S_interm_prim-Jensen')

In [152]:
interm_jen_left_accuracy

({102816: 2,
  103111: 2,
  103212: 1,
  103515: 1,
  104416: 2,
  104820: 2,
  105014: 1,
  105115: 2,
  105620: 1,
  107321: 2,
  107422: 1,
  108121: 1,
  108323: 1,
  108828: 1,
  109325: 2,
  111009: 1,
  111312: 1,
  111413: 2,
  112112: 1,
  112819: 1,
  113215: 1,
  114419: 2,
  116120: 2,
  116524: 1,
  117122: 2,
  118124: 2,
  118225: 2,
  121315: 2,
  121416: 2,
  121618: 1,
  122317: 2,
  122822: 1,
  123925: 2,
  124220: 1,
  124422: 1,
  125525: 2,
  127630: 2,
  130922: 1,
  131419: 2,
  131722: 1,
  131823: 2,
  132118: 2,
  133827: 1,
  133928: 2,
  135124: 1,
  135225: 1,
  135932: 2,
  136833: 1,
  137431: 2,
  138534: 2,
  139839: 2,
  140925: 1,
  148133: 2,
  148941: 1,
  150524: 1,
  151425: 1,
  151627: 1,
  152831: 1,
  153025: 1,
  153126: 2,
  153227: 2,
  153429: 1,
  154431: 2,
  154532: 1,
  154835: 1,
  155231: 2,
  155635: 1,
  156031: 1,
  157336: 2,
  158035: 1,
  158136: 1,
  161731: 1,
  162228: 2,
  163129: 1,
  163331: 1,
  167440: 1,
  168139: 1,

In [153]:
interm_jen_right_accuracy

({118225: 1,
  139839: 1,
  171633: 1,
  173435: 1,
  199453: 1,
  205725: 1,
  358144: 1,
  835657: 1,
  845458: 2},
 8,
 0.8888888888888888)

### Occipital Middle and Lunatus Sulcus

In [154]:
def oc_mid_lun_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = current_subject_hemi['S_oc-temp_lat']['posterior'].intersection(
           current_subject_hemi['S_subparietal']['inferior']).intersection(
               current_subject_hemi['S_occipital_ant']['superior'])

        sulci_found[current_subject]=sulcus_test

    return sulci_found

In [155]:
oc_mid_lun_check_left = oc_mid_lun_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
oc_mid_lun_check_right = oc_mid_lun_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [156]:
oc_mid_lun_prob_left = calculating_prob_of_sulcus_found(subject_ids, oc_mid_lun_check_left, 'Occipital_middle_and_lunatus_sulcus', 'S_oc_middle_and_Lunatus')
oc_mid_lun_prob_right = calculating_prob_of_sulcus_found(subject_ids, oc_mid_lun_check_right, 'Occipital_middle_and_lunatus_sulcus', 'S_oc_middle_and_Lunatus')

In [157]:
print(oc_mid_lun_prob_left, oc_mid_lun_prob_right)

(69, 572, 0.12062937062937062) (57, 572, 0.09965034965034965)


In [158]:
oc_mid_lun_left_accuracy = accuracy_check(subject_ids, oc_mid_lun_check_left, 'S_oc_middle_and_Lunatus')
oc_mid_lun_right_accuracy = accuracy_check(subject_ids, oc_mid_lun_check_right, 'S_oc_middle_and_Lunatus')

In [159]:
oc_mid_lun_left_accuracy

({103515: 3,
  105014: 2,
  107422: 3,
  110411: 2,
  112819: 2,
  121618: 3,
  122620: 1,
  131217: 1,
  133928: 1,
  134728: 2,
  135225: 1,
  135528: 2,
  143325: 4,
  151425: 2,
  153227: 2,
  155635: 2,
  156031: 2,
  157437: 2,
  161327: 2,
  162026: 3,
  162228: 4,
  164131: 3,
  167743: 2,
  172332: 4,
  180432: 1,
  180836: 1,
  183034: 1,
  185139: 4,
  185442: 2,
  187143: 2,
  195041: 3,
  196952: 3,
  197550: 3,
  204016: 2,
  208327: 2,
  211114: 2,
  211821: 3,
  217126: 2,
  245333: 1,
  280739: 3,
  280941: 2,
  293748: 3,
  333330: 2,
  334635: 2,
  355239: 3,
  365343: 3,
  366446: 1,
  397861: 3,
  412528: 2,
  419239: 3,
  436845: 4,
  453441: 3,
  500222: 2,
  536647: 2,
  568963: 3,
  654754: 3,
  695768: 1,
  713239: 3,
  731140: 4,
  753251: 2,
  773257: 3,
  825048: 2,
  841349: 3,
  844961: 1,
  856766: 1,
  872158: 3,
  887373: 2,
  896778: 2,
  965771: 2},
 12,
 0.17391304347826086)

In [160]:
oc_mid_lun_right_accuracy

({111716: 2,
  117122: 2,
  117324: 2,
  122822: 2,
  127630: 2,
  131217: 1,
  135932: 2,
  137431: 3,
  143325: 2,
  151223: 1,
  151930: 2,
  152427: 2,
  153227: 1,
  158540: 4,
  174437: 1,
  175439: 1,
  176542: 2,
  179346: 3,
  180129: 2,
  187143: 2,
  190031: 1,
  195041: 1,
  197550: 1,
  200311: 2,
  201111: 1,
  209329: 4,
  211215: 2,
  211821: 1,
  212419: 2,
  251833: 2,
  280739: 3,
  280941: 2,
  293748: 2,
  304020: 1,
  333330: 1,
  339847: 3,
  352738: 1,
  378756: 1,
  382242: 3,
  386250: 2,
  389357: 3,
  395251: 2,
  421226: 2,
  536647: 3,
  566454: 1,
  567961: 2,
  599469: 1,
  672756: 1,
  679568: 3,
  788674: 1,
  792766: 1,
  837560: 2,
  845458: 3,
  856766: 2,
  871964: 1,
  872158: 3,
  994273: 1},
 21,
 0.3684210526315789)

### Superior Occipital Sulcus

In [161]:
def sos_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, hemi):
    '''
    Input: subject_ids=list of all subjects' ids, used as keys in All_subject_and_hemisphere_sets=quadruple dictionary
    in pandas dataframe format, of subject, hemisphere, sulcus, set of sulci in relative position
    Function calculates relationship of how to find PrCS for each subject's left hemisphere
    Output: dictionary with subject id as key, and list of sulci as values.
    '''

    sulci_found = {}

    for current_subject in subject_ids:
        current_subject_hemi = All_subject_and_hemisphere_sets[current_subject][hemi]
        sulcus_test = current_subject_hemi['S_intrapariet_and_P_trans']['posterior'].intersection(
           current_subject_hemi['S_oc_middle_and_Lunatus']['superior']).intersection(
               current_subject_hemi['S_precentral-sup-part']['inferior'])

        sulci_found[current_subject]=sulcus_test

    return sulci_found

In [162]:
SOS_check_left = sos_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'left')
SOS_check_right = sos_prob_from_formula(subject_ids, All_subject_and_hemisphere_sets, 'right')

In [163]:
SOS_prob_left = calculating_prob_of_sulcus_found(subject_ids, SOS_check_left, 'Superior_occipital_sulcus', 'S_oc_sup_and_transversal')
SOS_prob_right = calculating_prob_of_sulcus_found(subject_ids, SOS_check_right, 'Superior_occipital_sulcus', 'S_oc_sup_and_transversal')

In [164]:
print(SOS_prob_left, SOS_prob_right)

(241, 572, 0.42132867132867136) (257, 572, 0.4493006993006993)


In [165]:
SOS_left_accuracy = accuracy_check(subject_ids, SOS_check_left, 'S_oc_sup_and_transversal')
SOS_right_accuracy = accuracy_check(subject_ids, SOS_check_right, 'S_oc_sup_and_transversal')

In [166]:
SOS_left_accuracy

({103111: 1,
  103212: 1,
  103414: 1,
  103818: 1,
  104012: 1,
  104820: 2,
  105216: 1,
  107018: 1,
  107321: 1,
  107422: 2,
  111009: 2,
  111312: 2,
  111716: 2,
  112112: 1,
  113215: 1,
  113619: 2,
  114419: 1,
  114924: 1,
  116524: 2,
  117122: 1,
  117324: 1,
  117930: 1,
  118124: 1,
  121618: 1,
  123117: 1,
  125424: 1,
  125525: 2,
  126931: 1,
  127933: 1,
  130922: 1,
  131621: 1,
  131722: 1,
  131823: 1,
  134324: 1,
  135528: 1,
  136833: 1,
  139435: 1,
  141826: 2,
  145531: 1,
  145632: 1,
  146533: 1,
  150625: 1,
  151728: 1,
  151930: 1,
  152427: 2,
  153227: 1,
  154431: 1,
  154734: 1,
  154835: 2,
  154936: 2,
  155231: 2,
  155635: 1,
  155938: 1,
  156233: 1,
  156637: 2,
  157336: 1,
  158843: 1,
  159239: 1,
  161327: 1,
  161630: 1,
  162026: 2,
  162329: 2,
  163129: 1,
  163331: 1,
  167238: 1,
  168341: 1,
  168947: 1,
  169141: 2,
  169444: 1,
  171330: 1,
  171431: 1,
  171633: 1,
  172029: 1,
  172938: 1,
  173334: 1,
  173940: 2,
  175035: 1,

In [167]:
SOS_right_accuracy

({102715: 1,
  103212: 1,
  103414: 1,
  104416: 1,
  105115: 1,
  105620: 1,
  106016: 1,
  106319: 1,
  106521: 1,
  107321: 1,
  107422: 1,
  108121: 2,
  109325: 1,
  111413: 1,
  112112: 1,
  113619: 1,
  113922: 1,
  114419: 1,
  114924: 1,
  115320: 1,
  116120: 1,
  116524: 1,
  117122: 1,
  117324: 1,
  117930: 1,
  118225: 1,
  121618: 1,
  122620: 1,
  123117: 1,
  123420: 1,
  123925: 1,
  125424: 1,
  125525: 1,
  126931: 1,
  127933: 1,
  131217: 1,
  131621: 1,
  131823: 1,
  132118: 1,
  133019: 1,
  133625: 1,
  133827: 1,
  133928: 1,
  134324: 1,
  135528: 1,
  136833: 1,
  137431: 1,
  138534: 2,
  140925: 1,
  141826: 1,
  143325: 1,
  148941: 1,
  150524: 1,
  150625: 1,
  151627: 1,
  152831: 1,
  153025: 1,
  153126: 1,
  153227: 1,
  153833: 1,
  154431: 1,
  154835: 1,
  154936: 1,
  155231: 1,
  156637: 1,
  157336: 1,
  158035: 1,
  158540: 1,
  158843: 1,
  159441: 1,
  162733: 1,
  163331: 2,
  165032: 1,
  168139: 1,
  169444: 1,
  171330: 2,
  171431: 1,